In [11]:
x = open("part1.txt") do f
    [l for l in eachline(f)]
end

3171-element Array{String,1}:
 "Before: [3, 0, 1, 3]"
 "15 2 1 3"            
 "After:  [3, 0, 1, 1]"
 ""                    
 "Before: [1, 3, 2, 0]"
 "11 2 2 0"            
 "After:  [4, 3, 2, 0]"
 ""                    
 "Before: [0, 3, 3, 1]"
 "14 3 2 0"            
 "After:  [3, 3, 3, 1]"
 ""                    
 "Before: [2, 3, 1, 3]"
 ⋮                     
 ""                    
 "Before: [0, 0, 2, 1]"
 "11 2 2 3"            
 "After:  [0, 0, 2, 4]"
 ""                    
 "Before: [2, 3, 1, 2]"
 "9 2 1 3"             
 "After:  [2, 3, 1, 1]"
 ""                    
 "Before: [0, 3, 1, 1]"
 "6 0 0 3"             
 "After:  [0, 3, 1, 0]"

In [53]:
addr(a, b , c, registers) = begin
    registers[c+1] = registers[a+1] + registers[b+1]
    registers
end

addi(a, b , c, registers) = begin
    registers[c+1] = registers[a+1] + b
    registers
end

mulr(a, b , c, registers) = begin
    registers[c+1] = registers[a+1] * registers[b+1]
    registers
end

muli(a, b , c, registers) = begin
    registers[c+1] = registers[a+1] * b
    registers
end

banr(a, b , c, registers) = begin
    registers[c+1] = registers[a+1] & registers[b+1]
    registers
end

bani(a, b , c, registers) = begin
    registers[c+1] = registers[a+1] & b
    registers
end

borr(a, b , c, registers) = begin
    registers[c+1] = registers[a+1] | registers[b+1]
    registers
end

bori(a, b , c, registers) = begin
    registers[c+1] = registers[a+1] | b
    registers
end

setr(a, b , c, registers) = begin
    registers[c+1] = registers[a+1]
    registers
end

seti(a, b , c, registers) = begin
    registers[c+1] = a
    registers
end

gtir(a, b , c, registers) = begin
    registers[c+1] = a > registers[b+1] ? 1 : 0
    registers
end

gtri(a, b , c, registers) = begin
    registers[c+1] =  registers[a+1] > b ? 1 : 0
    registers
end

gtrr(a, b , c, registers) = begin
    registers[c+1] = registers[a+1] > registers[b+1] ? 1 : 0
    registers
end

eqir(a, b , c, registers) = begin
    registers[c+1] = a == registers[b+1] ? 1 : 0
    registers
end

eqri(a, b , c, registers) = begin
    registers[c+1] =  registers[a+1] == b ? 1 : 0
    registers
end

eqrr(a, b , c, registers) = begin
    registers[c+1] = registers[a+1] == registers[b+1] ? 1 : 0
    registers
end

eqrr (generic function with 1 method)

In [54]:
const listfn = [addr, addi, mulr, muli, banr, bani, borr, bori, setr, seti, gtir, gtri, gtrr, eqir, eqri, eqrr]

16-element Array{Function,1}:
 addr
 addi
 mulr
 muli
 banr
 bani
 borr
 bori
 setr
 seti
 gtir
 gtri
 gtrr
 eqir
 eqri
 eqrr

In [55]:
before_registers = [parse.(Int, split(split(split(split(x,":")[2],"[")[2],"]")[1],",")) for x in x[1:4:end]];
after_registers = [parse.(Int, split(split(split(split(x,":")[2],"[")[2],"]")[1],",")) for x in x[3:4:end]];
instructions = [parse.(Int, split(x," ")) for x in x[2:4:end]];

In [70]:
# try instructions
try_instr(instructions, before_registers, after_registers) = begin
    sum([fn(instructions[2], instructions[3], instructions[4], copy(before_registers)) for fn in listfn] .== [after_registers])
end

try_instr (generic function with 1 method)

In [79]:
using StatsBase
countmap(try_instr.(instructions, before_registers, after_registers) .>= 3)

Dict{Bool,Int64} with 2 entries:
  false => 300
  true  => 493

In [84]:
# part 2 work out what each code is doing
# try instructions
try_instr2(instructions, before_registers, after_registers) = begin
    res = [fn(instructions[2], instructions[3], instructions[4], copy(before_registers)) for fn in listfn] .== [after_registers]
    (instructions[1], listfn[res])
end

try_instr2 (generic function with 1 method)

In [85]:
try_instr2(instructions[1], before_registers[1], after_registers[1])

(15, Function[addr, muli, bani, borr, bori, setr, gtir, gtrr, eqri])

In [97]:
res = try_instr2.(instructions, before_registers, after_registers)

793-element Array{Tuple{Int64,Array{Function,1}},1}:
 (15, [addr, muli, bani, borr, bori, setr, gtir, gtrr, eqri])                       
 (11, [addr, addi, mulr, muli])                                                     
 (14, [addi, mulr, borr, bori, seti])                                               
 (9, [muli, banr, bani, bori, setr, eqri])                                          
 (0, [banr, seti])                                                                  
 (8, [mulr, muli, borr, bori])                                                      
 (15, [addr, muli, bani, borr, bori, setr, gtir, gtrr, eqri])                       
 (15, [addr, muli, bani, borr, bori, setr, gtir, gtrr, eqri])                       
 (9, [muli, banr, bani, bori, setr, eqri])                                          
 (15, [addr, muli, bani, borr, bori, setr, gtir, gtrr, eqri])                       
 (4, [addi, bori])                                                                  
 (5, [mulr, 

In [118]:
res2 = [reduce(intersect, [y for (_, y) in filter(x->x[1] == opcode, res)]) for opcode in 0:15]

16-element Array{Array{Function,1},1}:
 [banr, seti]                                                                  
 [muli]                                                                        
 [bori]                                                                        
 [borr, bori, setr, seti]                                                      
 [addi, bori]                                                                  
 [mulr, muli, banr, bani, setr, gtir, gtri, gtrr, eqir, eqrr]                  
 [addr, addi, mulr, muli, banr, bani, borr, bori, setr, seti, gtir, gtri, gtrr]
 [addr, addi, borr, bori, gtir]                                                
 [borr, bori]                                                                  
 [muli, bani, bori, setr, eqri]                                                
 [bani]                                                                        
 [addr, addi, mulr, muli]                                                      
 

In [119]:
sures = filter(x->length(x[2]) == 1, zip(0:15, res2) |> collect)

3-element Array{Tuple{Int64,Array{Function,1}},1}:
 (1, [muli]) 
 (2, [bori]) 
 (10, [bani])

In [120]:
sures_fn =[sures[2][1] for sures in sures]

3-element Array{Function,1}:
 muli
 bori
 bani

In [129]:
res2 = [setdiff(res2, sures_fn) for res2 in res2]
sures = vcat(sures, filter(x->length(x[2]) == 1, zip(0:15, res2) |> collect))
sures_fn =[sures[2][1] for sures in sures]
sures

16-element Array{Tuple{Int64,Array{Function,1}},1}:
 (1, [muli]) 
 (2, [bori]) 
 (10, [bani])
 (4, [addi]) 
 (8, [borr]) 
 (13, [mulr])
 (14, [seti])
 (0, [banr]) 
 (3, [setr]) 
 (11, [addr])
 (7, [gtir]) 
 (9, [eqri]) 
 (15, [gtrr])
 (6, [gtri]) 
 (12, [eqir])
 (5, [eqrr]) 

In [132]:
# now i have the sures
opcode = Vector{Function}(undef, 16)
for (i, fn) in sures
    opcode[i+1] = fn[1]
end
opcode

16-element Array{Function,1}:
 banr
 muli
 bori
 setr
 addi
 eqrr
 gtri
 gtir
 borr
 eqri
 bani
 addr
 eqir
 mulr
 seti
 gtrr

In [138]:
xx = open("part2.txt") do f
    [parse.(Int, split(l," ")) for l in eachline(f)]
end

850-element Array{Array{Int64,1},1}:
 [1, 0, 0, 1] 
 [4, 1, 1, 1] 
 [14, 0, 0, 3]
 [14, 3, 2, 2]
 [12, 3, 2, 1]
 [1, 1, 2, 1] 
 [11, 1, 0, 0]
 [3, 0, 2, 1] 
 [14, 2, 2, 0]
 [9, 0, 2, 0] 
 [1, 0, 2, 0] 
 [11, 0, 1, 1]
 [3, 1, 2, 3] 
 ⋮            
 [14, 1, 1, 3]
 [13, 3, 0, 2]
 [1, 2, 3, 2] 
 [11, 2, 1, 1]
 [3, 1, 1, 3] 
 [14, 2, 2, 2]
 [14, 2, 2, 1]
 [14, 1, 3, 0]
 [3, 0, 2, 1] 
 [1, 1, 2, 1] 
 [11, 3, 1, 3]
 [3, 3, 2, 0] 

In [139]:
register= zeros(Int, 4)

4-element Array{Int64,1}:
 0
 0
 0
 0

In [140]:
for xx in xx
    opcode[xx[1] + 1](xx[2], xx[3], xx[4], register)
end
register

4-element Array{Int64,1}:
 445
   2
   2
 445

In [ ]:
register[1]